In [19]:
# let's import libraries
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
# this is the file you should've gotten from your repo
from utils import extract_dates, get_val_scores
from category_encoders import OrdinalEncoder, OneHotEncoder, TargetEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split

In [20]:
df = pd.read_csv("/Users/cameronlefevre/Data Science/coding/GA-DS-Class/Homework/Unit3/data/bikeshare.csv", parse_dates=['datetime'])

In [12]:
df = extract_dates(df)

/Users/cameronlefevre/Data Science/coding/GA-DS-Class/ClassMaterial/Unit3/Class15/utils.py:28: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[col_name] = getattr(df[col].dt, part)


In [21]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 00:00:00,Spring,0,0,Clear Skies,9.84,14.395,81,0.0,16
1,2011-01-01 01:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,40
2,2011-01-01 02:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,32
3,2011-01-01 03:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,13
4,2011-01-01 04:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,1


In [14]:
mod1 = GradientBoostingRegressor()

In [22]:
# this is the Scikit-Learn wrapper -- syntax is exactly the same as what we've used before
mod1.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [22]:
# let's go ahead and create a pipeline
pipe = make_pipeline(TargetEncoder(), GradientBoostingRegressor())

In [23]:
# let's do an initial checking of our validation set
X = df.drop(['datetime', 'count'], axis=1)
y = df['count']

In [25]:
# let's take a look at the arguments we can use for get_val_scores
?get_val_scores

In [24]:
# get an initial score for our model, with feature importances
scores = get_val_scores(pipe, X, y, return_test_score=True, randomize=False, stratify=False, use_kfold=False)

/Users/cameronlefevre/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/cameronlefevre/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [25]:
scores

{'validation_score': 0.1409684958556634, 'test_score': 0.05239435297468986}

In [28]:
# let's take a look at feature importances
feats = pd.DataFrame({ 'Importance': pipe.steps[1][1].feature_importances_, 'Column': X.columns})
feats.sort_values(by='Importance', ascending=False)

,Importance,Column
6,0.123426,goal
3,0.087642,main_category
2,0.082025,category
43,0.053122,deadline_history_days
41,0.050469,launched_month
30,0.041295,launched_is_month_start
39,0.041036,launched_hour
44,0.036439,launched_history_days
4,0.035812,currency
5,0.034382,country


In [29]:
# how long is your campaign?
df['duration'] = (df['deadline'] - df['launched']).dt.days

In [30]:
# get the average goal for each category
category_avgs = df.groupby('category')[['goal']].mean().reset_index().rename({'goal': 'category_goal_avg'}, axis=1)

In [31]:
# and merge it back in with the original dataset
df = df.merge(category_avgs, on='category')

,category,category_goal_avg
0,3D Printing,47096.558771
1,Academic,55365.812915
2,Accessories,14737.425295
3,Action,320007.524411
4,Animals,10788.377328
...,...,...
154,Woodworking,11655.184791
155,Workshops,9739.618038
156,World Music,28711.839405
157,Young Adult,40515.781588


In [33]:
# and for each campaign get the percent of category average
df['cat_goal_pct'] = df['goal'] / df['category_goal_avg']

In [34]:
# and our results
df[['goal', 'category_goal_avg', 'cat_goal_pct']].head()

,goal,category_goal_avg,cat_goal_pct
0,1533.95,5213.996468,0.294199
1,6060.97,5213.996468,1.162442
2,2000.00,5213.996468,0.383583
3,10000.00,5213.996468,1.917915
4,757.52,5213.996468,0.145286


In [35]:
# get the summary statistic
main_cats = df.groupby('main_category')[['goal']].mean().reset_index().rename({'goal': 'main_cat_goal_avg'}, axis=1)

In [36]:
# and merge it back in
df = df.merge(main_cats, on='main_category')

In [37]:
# and get the percent value
df['main_goal_pct'] = df['goal'] / df['main_cat_goal_avg']

In [38]:
# take a look
df[['goal', 'main_cat_goal_avg', 'main_goal_pct']].head()

,goal,main_cat_goal_avg,main_goal_pct
0,1533.95,22590.745149,0.067902
1,6060.97,22590.745149,0.268294
2,2000.00,22590.745149,0.088532
3,10000.00,22590.745149,0.442659
4,757.52,22590.745149,0.033532


In [39]:
# let's re-declare and run
X = df.drop(['deadline', 'launched', 'state'], axis=1)
y = df['state']

In [40]:
# refit
scores = get_val_scores(pipe, X, y, random_state=1985, stratify=True, use_kfold=False)

/Users/cameronlefevre/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/cameronlefevre/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:54:38] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [41]:
scores

{'validation_score': 0.7314797631299242}

In [42]:
# we'll get the number of campaigns launched during a particular week on Kickstarter
weekly_sums = df.groupby(['launched_year', 'launched_weekofyear'])[['ID']].count().reset_index().rename({'ID': 'Weekly_Campaigns'}, axis=1)

In [43]:
# and merge back in
df = df.merge(weekly_sums, on=['launched_year', 'launched_weekofyear'])

In [44]:
# and our final results
df.head()

,ID,name,category,main_category,currency,deadline,launched,state,country,goal,...,launched_month,launched_year,deadline_history_days,launched_history_days,duration,category_goal_avg,cat_goal_pct,main_cat_goal_avg,main_goal_pct,Weekly_Campaigns
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95,...,8,2015,2350,16658,58,5213.996468,0.294199,22590.745149,0.067902,1271
1,1403665672,Sentiments of a Sentimental Ol' Fool: A book t...,Poetry,Publishing,USD,2015-09-09,2015-08-10 17:36:06,failed,US,5000.00,...,8,2015,2320,16657,29,5213.996468,0.958957,22590.745149,0.221330,1271
2,1811433012,My first Poetry Book (Canceled),Poetry,Publishing,USD,2015-09-12,2015-08-13 05:08:15,failed,US,2000.00,...,8,2015,2323,16660,29,5213.996468,0.383583,22590.745149,0.088532,1271
3,75470413,The Circled Shore,Poetry,Publishing,USD,2015-09-11,2015-08-14 06:26:10,successful,US,3000.00,...,8,2015,2322,16661,27,5213.996468,0.575374,22590.745149,0.132798,1271
4,1053752807,The Armor Journal Magazine Goes In Print (Canc...,Nonfiction,Publishing,USD,2015-10-10,2015-08-11 07:45:32,failed,US,12000.00,...,8,2015,2351,16658,59,18906.372594,0.634707,22590.745149,0.531191,1271


In [45]:
# the daily totals
daily_sums = df.groupby(['launched_year', 'launched_dayofyear'])[['ID']].count().reset_index().rename({'ID': 'Daily_Campaigns'}, axis=1)

In [46]:
# merge it back in
df = df.merge(daily_sums, on=['launched_year', 'launched_dayofyear'])

In [47]:
# weekly totals for a particular category
cat_weekly_sums = df.groupby(['category', 'launched_year', 'launched_weekofyear'])[['ID']].count().reset_index().rename({'ID': 'Cat_Weekly_Campaigns'}, axis=1)

In [48]:
# merge them back in
df = df.merge(cat_weekly_sums, on=['category', 'launched_year', 'launched_weekofyear'])

In [49]:
# daily totals for a particular category
cat_daily_sums = df.groupby(['category', 'launched_year', 'launched_dayofyear'])[['ID']].count().reset_index().rename({'ID': 'Cat_Daily_Campaigns'}, axis=1)

In [50]:
# and merge back in
df = df.merge(cat_daily_sums, on=['category', 'launched_year', 'launched_dayofyear'])

In [51]:
# let's re-declare X & y
X = df.drop(['deadline', 'launched', 'state'], axis=1)
y = df['state']

In [52]:
# and fit again
scores = get_val_scores(pipe, X, y, random_state=1985, stratify=True, use_kfold=False)

/Users/cameronlefevre/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/cameronlefevre/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:57:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
